# Profiling dask vs numpy #

[Dask](http://dask.pydata.org/en/latest/index.html) is a powerful new tool for parallel processing in python. This notebooks does some benchmarking comparisons with regular numpy and suggests how astropy could be modified to take advantage of dask multithreading. (It should release the GIL more.) These tests were conducted on a server with 32 cores. 

In [ ]:
import numpy as np
import dask.array as da

## Create a large random numpy array ##

In [ ]:
Nz,Ny,Nx = (64,1024,2048)
shape = (Nz,Ny,Nx)
big_numpy_array = np.random.rand(*shape)

## Reduce operation using numpy ##

In [ ]:
%time (big_numpy_array**2).sum()

## Create a dask array from the numpy array ##
cs is the chunksize, so we use 64 different chunks of about 1 million elements

In [ ]:
cs = (1,Ny,Nx)
big_dask_array = da.from_array(big_numpy_array, chunks=cs)

In [ ]:
%time (big_dask_array**2).sum().compute()

We can see that we got some multithreaded acceleration, because the Wall time is less than the CPU time. This speedup is possible because the numpy ufuncs [release the global interpreter lock (GIL)](http://docs.scipy.org/doc/numpy-dev/reference/internals.code-explanations.html) at the cython level.

In [ ]:
# do the same thing using map_blocks
def map_func(b):
    return (b**2)

%time big_dask_array.map_blocks(map_func).sum().compute()

## FFT example ##
Here we also get speedup with numpy.fft because it evidently also releases the GIL.

In [ ]:
# try with fft
% time np.fft.fft2(big_numpy_array).sum()

In [ ]:
% time big_dask_array.map_blocks(np.fft.fft2).sum().compute()
# huge speedup!

## Gaussian Filter - scipy.ndimage ##
This does get multethreaded speeup.

In [ ]:
from scipy.ndimage import gaussian_filter

def filter_func(b):
    return gaussian_filter(b.squeeze(), 1)[np.newaxis,:,:]

# this does speed up because it releases the GIL
% time big_dask_array.map_blocks(filter_func).sum().compute()

## Gaussian Filter - astropy - upstream/master ##
Astropy convolution is understandably slower because it checks for missing data. But on top of this, it does NOT get multethreaded speeup because the [astropy convolution functions](https://github.com/astropy/astropy/blob/master/astropy/convolution/boundary_fill.pyx) do NOT release the GIL before their loops. As a result, it is nearly 60 times slow than ndimage when used with dask.

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.6-x86_64-3.5/')

from astropy.convolution import convolve, Gaussian2DKernel
import astropy
astropy.version

In [ ]:
def filter_func_ap(b):
    ker = Gaussian2DKernel(1)
    return convolve(b.squeeze(), ker, boundary='extend')[np.newaxis,:,:]

In [ ]:
%time big_dask_array.map_blocks(filter_func_ap).sum().compute()

In [ ]:
# test single threaded execution
%timeit filter_func_ap(big_numpy_array[0])

## switch to my fork ##

In [ ]:
%time big_dask_array.map_blocks(filter_func_ap).sum().compute()

In [ ]:
%timeit filter_func_ap(big_numpy_array[0])

In [ ]:
import numpy as np
import dask.array as da
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/')

from astropy.convolution import convolve, Gaussian2DKernel, convolve_dev
import astropy
print(astropy.version)
sys.stdout.flush()

Nz,Ny,Nx = (64,1024,2048)
shape = (Nz,Ny,Nx)
big_numpy_array = np.random.rand(*shape)

cs = (1,Ny,Nx)
big_dask_array = da.from_array(big_numpy_array, chunks=cs)

def filter_func_ap(b):
    ker = Gaussian2DKernel(1, x_size=11, y_size=11)
    return convolve(b.squeeze(), ker, boundary=None)[np.newaxis,:,:]
def filter_func_ap_dev(b):
    ker = Gaussian2DKernel(1, x_size=11, y_size=11)
    return convolve_dev(b.squeeze(), ker, boundary=None, n_threads=0)[np.newaxis,:,:]

In [ ]:
%time res = big_dask_array.map_blocks(filter_func_ap).compute()
%time res_dev = big_dask_array.map_blocks(filter_func_ap_dev).compute()

In [ ]:
d = res_dev - res
#print(d)
np.isclose(d, 0, atol=1e-8).all()

In [ ]:
res.shape

In [ ]:
%timeit filter_func_ap(big_numpy_array[0])

In [ ]:
810/760

In [ ]:
216/206


In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)

import astropy.convolution as astroconv
import numpy as np

iLength = 10000
jLength = iLength
image = np.random.random((iLength, jLength))

#image[50, 50] = np.nan

size = 11#1
ker = astroconv.Gaussian2DKernel(20,x_size=size,y_size=size)
#kerr = np.full((size,size), 10000)
#res = astroconv.convolve(image, ker, boundary=None, normalize_kernel=True, n_threads=1)






In [ ]:
%time res = astroconv.convolve(image, ker, boundary=None, normalize_kernel=True, n_threads=20)
#%time res_dev = astroconv.convolve_dev(image, ker, boundary=None, normalize_kernel=False)

In [ ]:
res.dtype

In [ ]:
res = astroconv.convolve(image, ker, boundary=None)
res_dev = astroconv.convolve_dev(image, ker, boundary=None)

In [ ]:
import matplotlib.pyplot as plt
#mport matplotlib
%matplotlib qt5

plt.imshow(res, cmap='gray')
plt.colorbar()

In [ ]:
#delta = res == res_dev
#delta.all()
d = res_dev - res
np.isclose(d, 0, atol=1e-8).all()

In [ ]:

import matplotlib.pyplot as plt
#mport matplotlib
%matplotlib qt5

plt.imshow(d, cmap='gray')
plt.colorbar()

In [ ]:
import sys
sys.path.insert(0, '/Users/jamienoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)

import astropy.convolution as astroconv
import numpy as np

iLength = 10000
jLength = iLength
image = np.random.random(iLength)

#image[50, 50] = np.nan

size = 11#1
ker = astroconv.Gaussian1DKernel(20,x_size=size)#,y_size=size)
#kerr = np.full((size,size), 10000)







In [ ]:
%time res = astroconv.convolve(image, ker, boundary='fill', normalize_kernel=False)
#%time res_dev = astroconv.convolve_dev(image, ker, boundary='fill', normalize_kernel=False, n_threads=0)

In [ ]:
2500/1024

In [ ]:
a = [0,12,3,4,5,6,7,8,9]
a[-3:]

In [ ]:
d[:20]

In [ ]:
res_dev.shape

In [ ]:
list(np.array(ker.shape)//2)

In [ ]:
pad = np.full(np.array(image.shape) + 2*(np.array(ker.shape)//2), fill_value=0.)


In [ ]:
pad_width = np.array(ker.shape)//2
image[pad_width[0]:-pad_width[0]].shape

In [ ]:
np.array((ker.shape//2))

In [ ]:
pad[pad_width[0]:-pad_width[0]] = image

In [ ]:
pad.shape

In [ ]:
pad[pad_width[0]:-pad_width[0]].shape

In [ ]:
pad_width[0]

In [ ]:
import ctypes
array_shape = np.array(image.shape, dtype=ctypes.c_size_t, order='C')
kernel_shape = np.array(ker.shape, dtype=ctypes.c_size_t, order='C')
pad_width = np.array(kernel_shape//2, dtyp) 
padded_array = np.full(array_shape + 2*pad_width, fill_value=0., dtype=float, order='C')
print(pad_width, padded_array.shape, padded_array[pad_width[0]:-pad_width[0]].shape)
padded_array[pad_width[0]:-pad_width[0]] = image        

In [ ]:
(int)pad_width[0].dtype

In [ ]:
image.dtype

In [ ]:
np.float_t

In [ ]:
a = np.array([1,2], dtype=np.float_)

In [ ]:
a.dtype

In [ ]:
import numpy as np
import dask.array as da
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/')

from astropy.convolution import convolve, Gaussian2DKernel, convolve_dev
import astropy
print(astropy.version)
sys.stdout.flush()

Nz,Ny,Nx = (10,10000,10000)
shape = (Nz,Ny,Nx)
big_numpy_array = np.random.rand(*shape)

cs = (1,Ny,Nx)
big_dask_array = da.from_array(big_numpy_array, chunks=cs)

def filter_func_ap(b):
    ker = Gaussian2DKernel(20, x_size=11, y_size=11)
    return convolve(b.squeeze(), ker, boundary=None)[np.newaxis,:,:]
def filter_func_ap_dev(b):
    ker = Gaussian2DKernel(20, x_size=11, y_size=11)
    return convolve_dev(b.squeeze(), ker, boundary=None, n_threads=1)[np.newaxis,:,:]

In [ ]:
#%time res = big_dask_array.map_blocks(filter_func_ap).compute()
%time res_dev = big_dask_array.map_blocks(filter_func_ap_dev).compute()

In [ ]:
d = res_dev2 - res
np.isclose(d, 0, atol=1e-8).all()

In [ ]:
import gc
gc.collect()

In [ ]:
import numpy as np
import dask.array as da
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/')

from astropy.convolution import convolve, Gaussian2DKernel, convolve_dev
import astropy
print(astropy.version)
sys.stdout.flush()

Nz,Ny,Nx = (10,10000,10000)
shape = (Nz,Ny,Nx)
big_numpy_array = np.random.rand(*shape)

res_dev2 = np.empty_like(big_numpy_array)
ker = Gaussian2DKernel(20, x_size=11, y_size=11)

In [ ]:
%time for i in range(10): res_dev2[i,:,:] = convolve_dev(big_numpy_array[i,:,:], ker, boundary=None, n_threads=1)

In [ ]:
ker = Gaussian2DKernel(20, x_size=10, y_size=11)
any(not axes_size % 2 for axes_size in ker.shape)

In [ ]:
ker = Gaussian2DKernel(20, x_size=10, y_size=11)
ker.array.ndim

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/')
import numpy as np
import astropy
print(astropy.version)
sys.stdout.flush()
from astropy.convolution import convolve
from numpy.testing import assert_array_almost_equal_nulp, assert_array_almost_equal, assert_almost_equal

from astropy.convolution import convolve, convolve_fft, convolve_models
from astropy.modeling import models, fitting
from astropy.utils.misc import NumpyRNGContext

def test_fitting_convolve_models(mode):
        """
        test that a convolve model can be fitted
        """
        b1 = models.Box1D()
        g1 = models.Gaussian1D()

        x = np.linspace(-5, 5, 99)
        fake_model = models.Gaussian1D(amplitude=10)
        with NumpyRNGContext(123):
            fake_data = fake_model(x) + np.random.normal(size=len(x))

        init_model = convolve_models(b1, g1, mode=mode, normalize_kernel=False)
        fitter = fitting.LevMarLSQFitter()
        fitted_model = fitter(init_model, x, fake_data)

        me = np.mean(fitted_model(x) - fake_data)
        assert_almost_equal(me, 0.0, decimal=2)

test_fitting_convolve_models('convolve')

In [ ]:
x = np.array([1., np.nan, 3.], dtype='>f8')
n = np.isnan(x)
z[n] = np.nan
z

In [ ]:
test_uniform_3_withnan('fill', 'interpolate'')

In [ ]:
x = np.array([1., np.nan, 3.], dtype='>f8')

y = np.array([1., 1., 1.], dtype='>f8')

z = convolve(x, y, boundary='fill', nan_treatment='interpolate',
             normalize_kernel=True,
             preserve_nan=False)
z

In [ ]:
from astropy.convolution.kernels import Gaussian1DKernel
import  astropy.convolution.tests.test_kernel_class
test = astropy.convolution.tests.test_kernel_class.TestKernels()
test.test_random_data(kernel_type=Gaussian1DKernel, width=9)

In [ ]:
import  astropy.convolution.tests.test_convolve_models as Test
test = Test.TestConvolve1DModels()
test.test_fitting_convolve_models('convolve')

In [ ]:
import numpy as np
x = np.array([1., 3, 3.], dtype='>f8')
y = np.array([1., 3, 4.], dtype='>f8')

In [ ]:
x.shape < y.shape

In [ ]:
for axis in x.shape:
    print(axis)

In [ ]:
any(x < y)

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/')
import numpy as np
import astropy
print(astropy.version)
sys.stdout.flush()
from astropy.convolution import convolve, 
x = np.array([1., 3, 3.], dtype='>f8')
y = np.array([1., 3, 4, ], dtype='>f8')

res = convolve(box, gauss)
print(res.array)

In [ ]:
box.shape

In [ ]:
1//2 < 1//2

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)
import faulthandler

faulthandler.enable()
import astropy.convolution as astroconv
import numpy as np

iLength = 11
jLength = iLength
image = np.random.random(iLength)#, jLength)

#image[50, 50] = np.nan

size = 41#iLength#11#1%time res = astroconv.convolve(image, ker, boundary='fill', normalize_kernel=False)

ker = astroconv.Gaussian1DKernel(20,x_size=size)

In [ ]:
%time res = astroconv.convolve(image, ker, boundary=None)#, n_threads=1)


In [ ]:
int(2 ** 9 - 1)

In [ ]:
import faulthandler
import astropy.convolution
import numpy as np

faulthandler.enable()

image = np.random.random(11)
ker = astropy.convolution.Gaussian1DKernel(20,x_size=41)

astropy.convolution.convolve(image, ker, boundary=None)

In [ ]:
nx = 11
wkx = 41//2
for i in range(wkx, nx - wkx):
    print(i)

In [ ]:
import os
os.environ['CFLAGS']

In [ ]:
from distutils.sysconfig import customize_compiler, get_config_var

get_config_var('CFLAGS')

In [ ]:
import distutils
distutils.sysconfig.get_config_var('CFLAGS')

In [ ]:
a = [1,2,3]
b = [2,3]
c = {'a':a, 'b':b}

In [ ]:
c['a']

In [ ]:
a = ''
os.path.join('.', *a.split('.'))

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy-helpers/build/lib/')
import astropy_helpers
print(astropy_helpers.version)

In [ ]:
from astropy_helpers.openmp_helpers import test_openmp_support, generate_openmp_enabled_py
from astropy_helpers.setup_helpers import _module_state, register_commands
#test_generate_openmp_enabled_py()

In [ ]:
def test_generate_openmp_enabled_py():

    register_commands('openmp_testing', '0.0', False)

    # Test file generation
    generate_openmp_enabled_py('')
    from openmp_enabled import is_openmp_enabled

    is_openmp_enabled = is_openmp_enabled()
    try:
        os.remove('openmp_enabled.py', )
    except:
        pass

    # test is_openmp_enabled()
    assert isinstance(is_openmp_enabled, bool)

In [ ]:
test_generate_openmp_enabled_py()

In [ ]:
pwd

In [ ]:
ls

In [ ]:
import openmp_enabled

In [ ]:
import  astropy_helpers.openmp_helpers as oh

In [ ]:
oh.get_openmp_flags()

In [ ]:
import sys
import numpy as np
from numpy.testing import assert_array_almost_equal_nulp, assert_array_almost_equal

sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)
from astropy.convolution import convolve
import pandas
wave = np.array((np.linspace(5000, 5100, 10)))
boxcar = 3
nonseries_result = convolve(wave, np.ones((boxcar,))/boxcar)

wave_series = pandas.Series(wave)
series_result  = convolve(wave_series, np.ones((boxcar,))/boxcar)

assert_array_almost_equal(nonseries_result, series_result)

In [ ]:
import numpy as np

class myarray(np.ndarray):
    
    def func1(self):
        print('you fool')
        


In [ ]:
a = np.array([1,1,1])
b = myarray(a)

In [ ]:
a

In [ ]:
b

In [ ]:
b.func1()

In [ ]:
aa = np.asanyarray(a)
bb=np.asanyarray(b)

In [ ]:
bb.func1()

In [ ]:
b is b3

In [ ]:
b3 = np.array(b, copy=False, subok=True)

In [ ]:
import numpy.ma as ma
x = np.array([1, 2, 3, -1, 5])
mx = ma.masked_array(x, mask=[0, 0, 0, 1, 0])
mxfilled = mx.filled(0)

In [ ]:
mx is mxfilled

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import CustomKernel
from scipy.signal import convolve as scipy_convolve
from astropy.convolution import convolve, convolve_fft


# Load the data from data.astropy.org
filename = get_pkg_data_filename('galactic_center/gc_msx_e.fits')
hdu = fits.open(filename)[0]

# Scale the file to have reasonable numbers
# (this is mostly so that colorbars don't have too many digits)
# Also, we crop it so you can see individual pixels
img = hdu.data[50:90, 60:100] * 1e5

kernel = CustomKernel([[-1,-1,-1], [-1, 8, -1], [-1,-1,-1]])

astropy_conv = convolve(img, kernel, normalize_kernel=False, nan_treatment='fill')

In [ ]:
from astropy.convolution import Kernel
isinstance(kernel,Kernel)

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)

import numpy as np
from astropy.convolution import convolve

kernel = [-1,-1,-1, -1, 8, -1, -1,-1,-1]
array = [1,2,3,4,5,6,7,8,9]

convolve(array,kernel, normalize_kernel=False, boundary='extend')

In [1]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)

astropy.test(package='convolution')

<module 'astropy.version' from '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/astropy/version.py'>
=========================================================== test session starts ============================================================
platform darwin -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0

Running tests with Astropy version 3.2.dev23197.
Running tests in build/lib.macosx-10.7-x86_64-3.6/astropy/convolution.

Date: 2018-11-01T16:14:53

Platform: Darwin-15.6.0-x86_64-i386-64bit

Executable: //anaconda/bin/python

Full Python Version: 
3.6.4 |Anaconda custom (64-bit)| (default, Mar 12 2018, 20:05:31) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]

encodings: sys: utf-8, locale: UTF-8, filesystem: utf-8
byteorder: little
float info: dig: 15, mant_dig: 15

Numpy: 1.15.1
Scipy: not available
Matplotlib: 2.0.2
h5py: 2.7.0
Pandas: not available
Cython: 0.27.3
astropy_helpers: 2.0.2
Using Astropy options: remote_data: none.

rootdir: /Users/jnoss/de

        '''
            Test that a unit kernel with a single element returns the same array
            '''
    
        x = np.array([1., 2., 3.], dtype='>f8')
    
        y = np.array([1.], dtype='>f8')
    
        z = convfunc(x, y, boundary=boundary)
    
>       np.testing.assert_allclose(z, x)
E       AssertionError: 
E       Not equal to tolerance rtol=1e-07, atol=0
E       
E       (mismatch 100.0%)
E        x: array([2.000000e+00, 3.000000e+00, 1.730598e-77])
E        y: array([1., 2., 3.])

../../astropy/build/lib.macosx-10.7-x86_64-3.6/astropy/convolution/tests/test_convolve.py:159: AssertionError
____________________________________________ TestConvolve1D.test_unity_1_none[convolve-extend] _____________________________________________

self = <astropy.convolution.tests.test_convolve.TestConvolve1D object at 0x11d9ccba8>, boundary = 'extend'
convfunc = <function convolve at 0x11cc05a60>

    @pytest.mark.parametrize(('convfunc', 'boundary',), BOUNDARIES_AND_CONVOLUTIONS)


        elif boundary == 'fill':
>           assert np.all(z == np.array([1., 4., 3.], dtype='>f8'))
E           assert False
E            +  where False = <function all at 0x10c4200d0>(array([4., 3., 3.]) == array([1., 4., 3.])
E            +    where <function all at 0x10c4200d0> = np.all
E               Use -v to get the full diff)

../../astropy/build/lib.macosx-10.7-x86_64-3.6/astropy/convolution/tests/test_convolve.py:195: AssertionError
__________________________________________________ TestConvolve1D.test_uniform_3[extend] ___________________________________________________

self = <astropy.convolution.tests.test_convolve.TestConvolve1D object at 0x11d9be6a0>, boundary = 'extend'

    @pytest.mark.parametrize(('boundary'), BOUNDARY_OPTIONS)
    def test_uniform_3(self, boundary):
        '''
            Test that the different modes are producing the correct results using
            a uniform kernel with three elements
            '''
    
        x = np.array([1., 0., 3.], dt

                                               NANHANDLING_OPTIONS,
                                               NORMALIZE_OPTIONS,
                                               PRESERVE_NAN_OPTIONS))
    def test_unity_3_withnan(self, boundary, nan_treatment,
                             normalize_kernel, preserve_nan):
        '''
            Test that a unit kernel with three elements returns the same array
            (except when boundary is None). This version includes a NaN value in
            the original array.
            '''
    
        x = np.array([1., np.nan, 3.], dtype='>f8')
    
        y = np.array([0., 1., 0.], dtype='>f8')
    
        z = convolve(x, y, boundary=boundary, nan_treatment=nan_treatment,
                     normalize_kernel=normalize_kernel,
                     preserve_nan=preserve_nan)
    
        if preserve_nan:
            assert np.isnan(z[1])
    
        x = np.nan_to_num(z)
        z = np.nan_to_num(z)
    
        if boundary is None:

        x = np.array([1., np.nan, 3.], dtype='>f8')
    
        y = np.array([1., 1., 1.], dtype='>f8')
    
        z = convolve(x, y, boundary=boundary, nan_treatment=nan_treatment,
                     normalize_kernel=normalize_kernel,
                     preserve_nan=preserve_nan)
    
        if preserve_nan:
            assert np.isnan(z[1])
    
        z = np.nan_to_num(z)
    
        # boundary, nan_treatment, normalize_kernel
        rslt = {
                (None, 'interpolate', True): [0, 2, 0],
                (None, 'interpolate', False): [0, 6, 0],
                (None, 'fill', True): [0, 4/3., 0],
                (None, 'fill', False): [0, 4, 0],
                ('fill', 'interpolate', True): [1/2., 2, 3/2.],
                ('fill', 'interpolate', False): [3/2., 6, 9/2.],
                ('fill', 'fill', True): [1/3., 4/3., 3/3.],
                ('fill', 'fill', False): [1, 4, 3],
                ('wrap', 'interpolate', True): [2, 2, 2],
                ('wrap', 

...

            Use only small kernel width, to make the test pass quickly.
            """
        if kernel_type == AiryDisk2DKernel and not HAS_SCIPY:
            pytest.skip("Omitting AiryDisk2DKernel, which requires SciPy")
        if not kernel_type == Ring2DKernel:
            kernel = kernel_type(3)
        else:
            kernel = kernel_type(3, 3 * 0.2)
    
        if kernel.dimension == 1:
            c1 = convolve_fft(delta_pulse_1D, kernel, boundary='fill', normalize_kernel=False)
            c2 = convolve(delta_pulse_1D, kernel, boundary='fill', normalize_kernel=False)
>           assert_almost_equal(c1, c2, decimal=12)
E           AssertionError: 
E           Arrays are not almost equal to 12 decimals
E           
E           (mismatch 32.09876543209876%)
E            x: array([-1.409462824231e-18,  3.144186300208e-18, -1.301042606983e-18,
E                   4.336808689942e-19,  3.469446951954e-18,  4.336808689942e-19,
E                   1.301042606983e-18,  6.071532165

            kernel = kernel_type(3, 3 * 0.2)
    
        if kernel.dimension == 1:
            c1 = convolve_fft(delta_pulse_1D, kernel, boundary='fill', normalize_kernel=False)
            c2 = convolve(delta_pulse_1D, kernel, boundary='fill', normalize_kernel=False)
>           assert_almost_equal(c1, c2, decimal=12)
E           AssertionError: 
E           Arrays are not almost equal to 12 decimals
E           
E           (mismatch 2.4691358024691397%)
E            x: array([ 1.098005159220e-17,  8.075327571284e-18,  3.051844252402e-18,
E                   5.859848183111e-18,  3.062423343959e-18,  3.465603023801e-18,
E                   2.071060801600e-19, -9.540979117872e-18,  1.387778780781e-17,...
E            y: array([0.000000000000e+000, 0.000000000000e+000, 0.000000000000e+000,
E                  0.000000000000e+000, 0.000000000000e+000, 0.000000000000e+000,
E                  0.000000000000e+000, 0.000000000000e+000, 0.000000000000e+000,...

../../astropy/build/lib.macosx-

>           assert_almost_equal(c1, c2, decimal=12)
E           AssertionError: 
E           Arrays are not almost equal to 12 decimals
E           
E           (mismatch 4.938271604938265%)
E            x: array([ 2.453269466693e-18, -7.879421846211e-19,  4.753067045046e-18,
E                   2.769509319286e-18, -2.946329633258e-18,  7.165743340798e-18,
E                   2.168404344971e-18,  2.255140518770e-17,  0.000000000000e+00,...
E            y: array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
E                  0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
E                  0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,...

../../astropy/build/lib.macosx-10.7-x86_64-3.6/astropy/convolution/tests/test_kernel_class.py:473: AssertionError
_____________________________________ TestKernels.test_discretize_modes[Trapezoid1DKernel-oversample] ______________________________________

self = <astropy.convolution.tests.test_kernel_

E                  -5.963111948671e-19,  4.336808689942e-19,  4.513545557487e-19,
E                   2.710505431214e-20,  5.692061405549e-19, -2.407412430484e-35,...
E            y: array([ 0.000000000000e+00,  0.000000000000e+00,  0.000000000000e+00,
E                   0.000000000000e+00,  0.000000000000e+00,  0.000000000000e+00,
E                   0.000000000000e+00,  0.000000000000e+00,  0.000000000000e+00,...

../../astropy/build/lib.macosx-10.7-x86_64-3.6/astropy/convolution/tests/test_kernel_class.py:473: AssertionError
_____________________________________ TestKernels.test_discretize_modes[MexicanHat1DKernel-oversample] _____________________________________

self = <astropy.convolution.tests.test_kernel_class.TestKernels object at 0x11da5fba8>
kernel_type = <class 'astropy.convolution.kernels.MexicanHat1DKernel'>, mode = 'oversample'

    @pytest.mark.parametrize(('kernel_type', 'mode'), list(itertools.product(KERNEL_TYPES, MODES)))
    def test_discretize_modes(self, kernel_

1

In [ ]:
import numpy as np

a = np.array([-1, -1, -1, -1, 8, -1, -1, -1, -1])
a.sum()

In [ ]:
y = ma.array([-1, -1, -1, -1, 8, -1, -1, -1, -1], mask=[0, 0, 0, 0, 1, 0, 0, 0, 0,], fill_value=100)

In [ ]:
import numpy.ma as ma
y.view(ma.MaskedArray)

In [ ]:
z = y.filled()

In [ ]:
z

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)

#astropy.test(package='convolution')
import numpy as np
import numpy.ma as ma
from astropy.convolution import convolve
x = [1, 2, 3, 4, 5, 6, 7, 8, 9]
y = ma.array([-1, -1, -1, -1, 8, -1, -1, -1, -1], mask=[1, 0, 0, 0, 0, 0, 0, 0, 0], fill_value=0.)
np.ma.is_masked(y)
#y.sum()

#convolve(x,y,normalize_kernel=False, boundary=None)

In [ ]:
y = ma.array([-1, -1, -1, -1, 7, -1, -1, -1, -1], mask=[1, 0, 0, 0, 0, 0, 0, 0, 0])
z = y.filled(dtype=float)

In [ ]:
isinstance(y.astype(float), float)

In [ ]:
y.astype(float).dtype

In [ ]:
y.filled().sum()

In [ ]:
y.sum()

In [ ]:
isinstance(y, np.ndarray)

In [ ]:
np.ma.is_masked(y)

In [ ]:
y[1] = np.nan

In [ ]:
import numpy as np
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)
from astropy.convolution import convolve


x = (1, np.nan, 3, 4, 5, 6, 7, 8, 9)
y = (-1, 2, -1, -1, 8, -1, -1, -1, -1)
convolve(x,y,normalize_kernel=False, boundary=None, nan_treatment='fill')

In [ ]:
np.isclose(sum(x), 0, atol=1e-8)

In [ ]:
y.mask.any()

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)
import astropy.convolution

import numpy as np
import copy

def convolve(x, y, *args, **kargs):
    x_hash = x.__hash__()
    y_hash = copy.deepcopy(y.__hash__)

    ret = convolve_actual(x, y, *args, **kargs)

    assert(x_hash == x.__hash__)
    assert(y_hash == y.__hash__)

    return ret

x = np.array([1, 2, 2, 4, 5, 6, 7, 8, 9])

y = [-1, -1, -1, -1, 8, -1, -1, -1, 0]

convolve(x,y,normalize_kernel=False, boundary=None, nan_treatment='fill')

In [ ]:
print(hash(x))

In [ ]:
x == x

In [ ]:
x

In [ ]:
np.nan == np.nan

In [ ]:
import hashlib
hashlib.sha1(x).hexdigest()

In [ ]:
x_c = copy.deepcopy(x)
hashlib.sha1(x_c).hexdigest()

In [ ]:
x_c.flags['WRITEABLE']=False
hashlib.sha1(x_c).hexdigest()

In [ ]:
x_c == x

In [ ]:
import collections
im_x = collections.namedtuple("Immutable", x)


In [ ]:

immutable_types = set((int, str))

class Immutable(object):
    def __init__(self, value):
        self._value = value
        #super(Immutable, self).__setattr__('_value', value)

    def __getattribute__(self, name):
        if name == '_value':
            return super(Immutable, self).__getattribute__(name)
        v = getattr(self._value, name)
        return v if v.__class__ in immutable_types else freeze(v)

    def __setattr__(self, name, value):
        if name == '_value': super(Immutable, self).__setattr__(name, value)
        else: raise Exception("Can't modify frozen object")

    #__slots__ = []

def freeze(object):
  return Immutable(object)

In [ ]:
import numpy as np
x = np.array([1, 2, 2, 4, 5, 6, 7, np.nan, np.nan])
print(np.isnan(x))

In [ ]:
x = ((9, 3, 1), (1,1,1), (1,1,1))
np.asanyarray(x, dtype=float)

In [ ]:
def t1():
    print('t1')
    
class T:
    @staticmethod
    def t1():
        print('t2')
        
    def test(self):
        self.t1()

a = T()
a.test()

In [ ]:
import astropy
from astropy.convolution import convolve_fft
x = [1,2,3,4,5,6]
y = [1,2,3]
convolve_fft(x,y)


In [ ]:
a = {'a':[1,3], 'b':2}
list(a.values())

In [ ]:
def foo(a,b,c):
    pass

foo.__code__.co_varnames

In [ ]:
inter = set(foo.__code__.co_varnames).intersection(set(a))

In [ ]:
a[inter]

In [ ]:
nl = []
for arg in inter:
    nl.append(a[arg])

In [ ]:
nl

In [ ]:
[a[arg] for arg in inter]

In [ ]:
import itertools

a = ['a','b','c']
b = ['d','e','f']
print(list(itertools.product(a,a)))

In [ ]:
VALID_DTYPES = []
for dtype_array in ['>f4', '<f4', '>f8', '<f8']:
    for dtype_kernel in ['>f4', '<f4', '>f8', '<f8']:
        VALID_DTYPES.append((dtype_array, dtype_kernel))
VALID_DTYPES

In [ ]:
t = ['>f4', '<f4', '>f8', '<f8']
list(itertools.product(t,t))

In [ ]:
list(itertools.product(t,a))

In [ ]:
t.extend(a)
t

In [ ]:
import sys
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)

from astropy.convolution.tests.convolution_test_classes import Parameterizer
from astropy.convolution.tests.test_convolve import TestConvolve1D

import itertools
obj = TestConvolve1D()

parameter_space = obj.parameter_space
func_arg_names = obj.test_dtype.__code__.co_varnames
argnames = list(set(func_arg_names) & set(parameter_space.keys()))


value_space = [parameter_space[arg] for arg in argnames]


para = itertools.product(*value_space)
list(para)

In [ ]:
list(itertools.product(value_space, repeat= len(argnames)))

In [ ]:
list(argnames)

In [ ]:
import sys
import numpy as np
from numpy.testing import assert_array_almost_equal_nulp, assert_array_almost_equal
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
print(astropy.version)
inlist = [1, 4, 5, 6, 5, 7, 8]
x = np.array(inlist)
y = [0.2, 0.6, 0.2]
z = astropy.convolution.convolve(x, y, boundary='fill')
z

In [ ]:
import itertools

VALID_DTYPES = ('>f4', '<f4', '>f8', '<f8')
VALID_DTYPE_MATRIX = list(itertools.product(VALID_DTYPES, VALID_DTYPES))

In [ ]:
VALID_DTYPE_MATRIX

In [ ]:
import sys
import numpy as np
import numpy.ma as ma
from numpy.testing import assert_array_almost_equal_nulp, assert_array_almost_equal
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
from astropy.convolution import convolve
print(astropy.version)
x = [3, 5, 7, 11, 13]
#x = ma.array([3, 5, 7, 11, 13], mask=[0, 0, 1, 0, 0], fill_value=0.)#,  dtype='>f8')
y = np.array([1., 1., 1.], dtype='>f8')

convolve(x, y, preserve_nan=False)

In [ ]:
8/3

In [ ]:
len("             HSTIO error 114:  Filename /Users/jnoss/dev/test/issuexxx/jd0q14cwq_flc.fits EXTNAME SCI EXTVER 3 CFITSIO status 301ffopen could not move to the specified extension: extension with EXTNAME = SCI,           and with EXTVERS = 3, doesn't exist or couldn't be opened.")

In [ ]:
a = [1,1,1]
isinstance(a, (list, tuple))

In [ ]:
import sys
import numpy as np
import numpy.ma as ma
from numpy.testing import assert_array_almost_equal_nulp, assert_array_almost_equal
sys.path.insert(0, '/Users/jnoss/dev/astropy/build/lib.macosx-10.7-x86_64-3.6/')
import astropy
from astropy.openmp_enabled import is_openmp_enabled
is_openmp_enabled()

In [ ]:
import subprocess
from distutils.errors import CompileError, LinkError
try:
    subprocess.check_output("/Users/jnoss/Desktop/a.out")
except (CompileError, LinkError):
    raise
except subprocess.CalledProcessError as error:
        print(error)
print("still here")

In [ ]:
word = "blah"
print("I said", word)
